In [2]:
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, clear_output, Markdown
import pandas as pd

In [2]:
print(dir(widgets))

['Accordion', 'Audio', 'BoundedFloatText', 'BoundedIntText', 'Box', 'Button', 'ButtonStyle', 'CallbackDispatcher', 'Checkbox', 'Color', 'ColorPicker', 'Controller', 'CoreWidget', 'DOMWidget', 'DatePicker', 'Datetime', 'Dropdown', 'FloatLogSlider', 'FloatProgress', 'FloatRangeSlider', 'FloatSlider', 'FloatText', 'GridBox', 'HBox', 'HTML', 'HTMLMath', 'Image', 'IntProgress', 'IntRangeSlider', 'IntSlider', 'IntText', 'Label', 'Layout', 'NumberFormat', 'Output', 'Password', 'Play', 'RadioButtons', 'Select', 'SelectMultiple', 'SelectionRangeSlider', 'SelectionSlider', 'SliderStyle', 'Style', 'Tab', 'Text', 'Textarea', 'ToggleButton', 'ToggleButtons', 'ToggleButtonsStyle', 'VBox', 'Valid', 'ValueWidget', 'Video', 'Widget', '__builtins__', '__cached__', '__doc__', '__file__', '__jupyter_widgets_base_version__', '__jupyter_widgets_controls_version__', '__loader__', '__name__', '__package__', '__path__', '__protocol_version__', '__spec__', '__version__', '_handle_ipython', '_version', 'dlink', 

In [6]:
df = pd.read_parquet('/Users/bin_wang/Documents/project-alice/data/04_features/features_training.parquet')

In [7]:
df.head(1)

,A_AAC,C_AAC,D_AAC,E_AAC,F_AAC,G_AAC,H_AAC,I_AAC,K_AAC,L_AAC,...,g7.g7.g7.gap0_KSCTriad,cell_line,peptide,norm_value,penetration,sequence,source,id,CL1,CL2
0,0.090909,0.0,0.030303,0.0,0.0,0.151515,0.030303,0.030303,0.181818,0.0,...,0.0,CL1,0762-SpyTag,3.759519,1.0,RKRKPGKHRGNRIGKRSDRVGNARPKGSARRKA,Internal,0,1,0


## use observe to create relationship between widgets

In [111]:
dropdown = widgets.Dropdown(options=df.columns)

def dropdown_change(change):
    print(df[change['new']])

# register observer
dropdown.observe(dropdown_change, names='value')


In [119]:
dropdown

Dropdown(index=4, options=('A_AAC', 'C_AAC', 'D_AAC', 'E_AAC', 'F_AAC', 'G_AAC', 'H_AAC', 'I_AAC', 'K_AAC', 'L…

0      0.151515
1      0.148148
2      0.153846
3      0.142857
4      0.142857
5      0.000000
6      0.000000
7      0.000000
8      0.000000
9      0.000000
10     0.000000
11     0.000000
12     0.000000
13     0.000000
14     0.000000
15     0.000000
16     0.000000
17     0.000000
18     0.000000
19     0.000000
20     0.111111
21     0.021277
22     0.272727
23     0.190476
24     0.150000
25     0.068966
26     0.000000
27     0.000000
28     0.285714
29     0.080000
         ...   
328    0.000000
329    0.096774
330    0.033333
331    0.000000
332    0.090909
333    0.000000
334    0.230769
335    0.035714
336    0.090909
337    0.217391
338    0.000000
339    0.021739
340    0.064516
341    0.043478
342    0.080000
343    0.080000
344    0.043478
345    0.000000
346    0.085714
347    0.000000
348    0.000000
349    0.179487
350    0.100000
351    0.090909
352    0.024390
353    0.000000
354    0.000000
355    0.033333
356    0.000000
357    0.000000
Name: G_AAC, Length: 358

## use @interact to auto create widgets on function parameters

this is the best version of API!

base function is intact

in decorator, you indicate values and what widget to use for this value

In [109]:
@interact(
        col=[x for x in df.columns if '_AAC' in x], 
        val=widgets.Text(value='0.1'),
        val2=(0,10,0.1),
        val3={'one': 1, 'two': 2},
        val4=1)
def filter_data(col, val, val2, val3) -> pd.DataFrame:
    return df[df[col] > float(val)]


interactive(children=(Dropdown(description='col', options=('A_AAC', 'C_AAC', 'D_AAC', 'E_AAC', 'F_AAC', 'G_AAC…

In [120]:
@widgets.interact
def scatter_plot(
        x=df.columns, 
        y=df.columns,
        color=['red','green','blue']):
    df.plot(x=x, y=y, kind='scatter', c=color)
    print(df[[x,y]].corr())


interactive(children=(Dropdown(description='x', options=('A_AAC', 'C_AAC', 'D_AAC', 'E_AAC', 'F_AAC', 'G_AAC',…

### experiment: dropdown of internal/external, display file list

In [104]:
source_dropdown = widgets.Dropdown(options=['external', 'internal'])
filelist_dropdown = widgets.Dropdown(options=['cpp924.txt'])
show_button = widgets.Button(description='Show file...')
output = widgets.Output()

In [105]:
def source_change(change):
    if change['new'] == 'internal':
        filelist_dropdown.options = ['peptidesToGet.txt.core.fa', 'qb_data_analysis_final.xlsx']
    else:
        filelist_dropdown.options = ['cpp924.txt']

source_dropdown.observe(source_change, names='value')


In [106]:
def button_click(button):
    path = '/Users/bin_wang/Documents/project-alice/data/01_raw'
    filename = filelist_dropdown.value
    output.clear_output()
    with output:
        if 'txt' in filename or '.fa' in filename:
            print(pd.read_csv(f'{path}/{filename}').head(10))
        else:
            print(pd.read_excel(f"{path}/{filename}").head(10))
    
show_button.on_click(button_click)

In [107]:
display(source_dropdown, filelist_dropdown, show_button, output)

Dropdown(options=('external', 'internal'), value='external')

Dropdown(options=('cpp924.txt',), value='cpp924.txt')

Button(description='Show file...', style=ButtonStyle())

Output()